形態素を表すクラスMorphを実装せよ．このクラスは表層形（surface），基本形（base），品詞（pos），品詞細分類1（pos1）をメンバ変数に持つこととする．さらに，CaboChaの解析結果（neko.txt.cabocha）を読み込み，各文をMorphオブジェクトのリストとして表現し，3文目の形態素列を表示せよ．

In [1]:
class Morph:
    def __init__(self, surface, base, pos, pos1):
        self.surface = surface
        self.base = base
        self.pos = pos
        self.pos1 = pos1
    def __str__(self):
        return self.surface

In [2]:
import re
sentences = []
sentence = []
for line in open('data/neko.txt.cabocha'):
    x = re.search('(?P<surface>[^\t]*?)\t(?P<pos>[^,]*),(?P<pos1>[^,]*),[^,]*,[^,]*,[^,]*,[^,]*,(?P<base>[^,]*).*',line)
    if x:
        morph = Morph(x.group('surface'), x.group('base'), x.group('pos'), x.group('pos1'))
#         print(morph)
        sentence.append(morph)
    elif line == 'EOS\n' and not sentence == []:
        sentences.append(sentence)
        sentence = []

In [3]:
print([[str(y) for y in x] for x in sentences[:10]])

[['一'], ['\u3000', '吾輩', 'は', '猫', 'で', 'ある', '。'], ['名前', 'は', 'まだ', '無い', '。'], ['\u3000', 'どこ', 'で', '生れ', 'た', 'か', 'とんと', '見当', 'が', 'つか', 'ぬ', '。'], ['何', 'でも', '薄暗い', 'じめじめ', 'し', 'た', '所', 'で', 'ニャーニャー', '泣い', 'て', 'いた事', 'だけ', 'は', '記憶', 'し', 'て', 'いる', '。'], ['吾輩', 'は', 'ここ', 'で', '始め', 'て', '人間', 'という', 'もの', 'を', '見', 'た', '。'], ['しかも', 'あと', 'で', '聞く', 'と', 'それ', 'は', '書生', 'という', '人間', '中', 'で', '一番', '獰悪', 'な', '種族', 'で', 'あっ', 'た', 'そう', 'だ', '。'], ['この', '書生', 'という', 'の', 'は', '時々', '我々', 'を', '捕え', 'て', '煮', 'て', '食う', 'という', '話', 'で', 'ある', '。'], ['しかし', 'その', '当時', 'は', '何', 'という', '考', 'も', 'なかっ', 'た', 'から', '別段', '恐し', 'いとも', '思わ', 'なかっ', 'た', '。'], ['ただ', '彼', 'の', '掌', 'に', '載せ', 'られ', 'て', 'スー', 'と', '持ち上げ', 'られ', 'た', '時', '何だか', 'フワフワ', 'し', 'た', '感じ', 'が', 'あっ', 'た', 'ばかり', 'で', 'ある', '。']]


In [4]:
class Chunk:
    def __init__(self, morphs, dst, srcs):
        self.morphs = morphs
        self.dst = dst
        self.srcs = srcs
    def __str__(self):
        ret = ''.join([str(x) for x in self.morphs])
        return ret

In [5]:
import re
sentences = []#sentence の list
sentence = []#Chunk の list
for line in open('data/neko.txt.cabocha'):
    x = re.search('(?P<surface>.*?)\t(?P<pos>[^,]*),(?P<pos1>[^,]*),[^,]*,[^,]*,[^,]*,[^,]*,(?P<base>[^,]*).*',line)
    if x:
        morph = Morph(x.group('surface'), x.group('base'), x.group('pos'), x.group('pos1'))
        sentence[-1].morphs.append(morph)
        continue
    y = re.search('\*\ (?P<number>[0-9]+)\ (?P<dst>-?[0-9]*)D.*', line)
    if y:
        dst = y.group('dst')
        srcs = [i for i, j in enumerate(sentence) if j.dst == y.group('number')]
        chunk = Chunk([], dst, srcs)
        sentence.append(chunk)
    elif line == 'EOS\n' and not sentence == []:
        sentences.append(sentence)
        sentence = []

In [6]:
s = sentences[2]
print([str(i) + '->' + str(s[int(i.dst)])  if i.dst != '-1' else str(i) for i in s])

['名前は->無い。', 'まだ->無い。', '無い。']


## 42. 係り元と係り先の文節の表示
係り元の文節と係り先の文節のテキストをタブ区切り形式ですべて抽出せよ．ただし，句読点などの記号は出力しないようにせよ．

In [7]:
with open("data/file42.txt", "w") as f:
    for sentence in sentences:
        for chunk in sentence:
            p = [str(sentence[int(i)] for i in chunk.srcs)] + [str(chunk)]
            p = p + [str(sentence[int(chunk.dst)])] if chunk.dst != '-1' else p
            f.write('\t'.join(list(p)) +  '\n')

In [8]:
with open('data/file42.txt', 'r') as r:
    print(''.join(r.readlines()[:10]))

<generator object <genexpr> at 0x112086f68>	一
<generator object <genexpr> at 0x112086fc0>	　	猫である。
<generator object <genexpr> at 0x112086f10>	吾輩は	猫である。
<generator object <genexpr> at 0x112086f10>	猫である。
<generator object <genexpr> at 0x112086fc0>	名前は	無い。
<generator object <genexpr> at 0x112086f10>	まだ	無い。
<generator object <genexpr> at 0x112086f10>	無い。
<generator object <genexpr> at 0x112086fc0>	　どこで	生れたか
<generator object <genexpr> at 0x112086e60>	生れたか	つかぬ。
<generator object <genexpr> at 0x112086eb8>	とんと	つかぬ。



In [9]:
with open('data/file43.txt', 'a') as f:
    for sentence in sentences:
        for chunk in sentence:
            if not [0 for i in chunk.morphs if i.pos == '名詞']:
                continue
            chun2 = sentence[int(chunk.dst)]
            if not [0 for i in chun2.morphs if i.pos == '動詞']:
                continue
            f.write(str(chunk) + '\t' + str(chun2) + '\n')

In [10]:
with open('data/file43.txt', 'r') as r:
    print(''.join(r.readlines()[:10]))

　どこで	生れたか
見当が	つかぬ。
所で	泣いて
ニャーニャー	泣いて
いた事だけは	記憶している。
記憶している。	記憶している。
吾輩は	見た。
ここで	始めて
ものを	見た。
あとで	聞くと



## 44. 係り受け木の可視化
与えられた文の係り受け木を有向グラフとして可視化せよ．可視化には，係り受け木をDOT言語に変換し，Graphvizを用いるとよい．また，Pythonから有向グラフを直接的に可視化するには，pydotを使うとよい．

In [11]:
import networkx as nx
s = sentences[4]
G = nx.Graph()
for chunk in s:
    G.add_node(chunk, label=str(chunk))
for chunk in s[:-1]:
    G.add_edge(chunk, s[int(chunk.dst)])
g = nx.nx_agraph.to_agraph(G)
g.draw(f"data/file44.pdf",format='pdf', prog='dot')

ImportError: ('requires pygraphviz ', 'http://pygraphviz.github.io/')

## 45. 動詞の格パターンの抽出
今回用いている文章をコーパスと見なし，日本語の述語が取りうる格を調査したい． 動詞を述語，動詞に係っている文節の助詞を格と考え，述語と格をタブ区切り形式で出力せよ． ただし，出力は以下の仕様を満たすようにせよ．

動詞を含む文節において，最左の動詞の基本形を述語とする
述語に係る助詞を格とする
述語に係る助詞（文節）が複数あるときは，すべての助詞をスペース区切りで辞書順に並べる
「吾輩はここで始めて人間というものを見た」という例文（neko.txt.cabochaの8文目）を考える． この文は「始める」と「見る」の２つの動詞を含み，「始める」に係る文節は「ここで」，「見る」に係る文節は「吾輩は」と「ものを」と解析された場合は，次のような出力になるはずである．

始める  で
見る    は を
このプログラムの出力をファイルに保存し，以下の事項をUNIXコマンドを用いて確認せよ．

コーパス中で頻出する述語と格パターンの組み合わせ
「する」「見る」「与える」という動詞の格パターン（コーパス中で出現頻度の高い順に並べよ）


In [12]:
with open('data/file45.txt', 'w') as f:
    for sentence in sentences:
        for chunk in sentence:
            if chunk.morphs[-1].surface == '、':
                chunk.morphs = chunk.morphs[:-1]
            verbs = [i.base for i in chunk.morphs if i.pos == '動詞']
            if verbs != []:
                verb = verbs[0]
                particles =  [sentence[i].morphs[-1].base for i in list(map(int,chunk.srcs)) if sentence[i].morphs[-1].pos == '助詞']
                if particles != []:
                    f.write(verb + '\t' + ' '.join(particles) + '\n')

In [13]:
!sort data/file45.txt | uniq -c | sort -r | head -n 10

 705 云う	と
 467 する	を
 343 思う	と
 206 ある	が
 203 なる	に
 195 する	に
 182 見る	て
 161 する	と
 118 する	が
 100 見る	を
sort: Broken pipe


In [14]:
!grep  '^与える\t' data/file45.txt | sort | uniq -c | sort -r

   4 与える	に を
   2 与える	て に を
   1 与える	けれども は を
   1 与える	として か
   1 与える	が は と て に を
   1 与える	は て に を に
   1 与える	は て に を
   1 与える	と は て を
   1 与える	て は に を
   1 与える	は は も
   1 与える	で に を
   1 与える	も を
   1 与える	ば を
   1 与える	が を


## 46. 動詞の格フレーム情報の抽出
45のプログラムを改変し，述語と格パターンに続けて項（述語に係っている文節そのもの）をタブ区切り形式で出力せよ．45の仕様に加えて，以下の仕様を満たすようにせよ．

項は述語に係っている文節の単語列とする（末尾の助詞を取り除く必要はない）
述語に係る文節が複数あるときは，助詞と同一の基準・順序でスペース区切りで並べる 「吾輩はここで始めて人間というものを見た」という例文（neko.txt.cabochaの8文目）を考える． この文は「始める」と「見る」の２つの動詞を含み，「始める」に係る文節は「ここで」，「見る」に係る文節は「吾輩は」と「ものを」と解析された場合は，次のような出力になるはずである．

始める で ここで

見る は を 吾輩は ものを

In [16]:
with open('data/file46.txt', 'w') as f:
    for sentence in sentences:
        for chunk in sentence:
            if chunk.morphs[-1].surface == '、':
                chunk.morphs = chunk.morphs[:-1]
            verbs = [i.base for i in chunk.morphs if i.pos == '動詞']
            if verbs != []:
                verb = verbs[0]
                units = [sentence[i] for i in list(map(int,chunk.srcs)) if sentence[i].morphs[-1].pos == '助詞']
                particles = [i.morphs[-1].base for i in units]
                if particles != []:
                    f.write(verb + '\t' + ' '.join(particles) + '\t' + ' '.join([str(i) for i in units]) + '\n')

In [17]:
i = 0
for line in open('data/file46.txt'):
    print(line, end='')
    i = i + 1
    if i == 10:
        break

生れる	で	　どこで
つく	か が	生れたか 見当が
泣く	で	所で
する	て は	泣いて いた事だけは
始める	で	ここで
見る	は を	吾輩は ものを
聞く	で	あとで
捕える	を	我々を
煮る	て	捕えて
食う	て	煮て
